# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.3 Epdemics

#### Exercise 2.9: Simulate an epidemic outbreak

In [59]:
import epidemics_helper
import json
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt

In [42]:
# Load the data of nyc network
with open('../data/nyc_augmented_network.json') as data_file:    
    data = json.load(data_file)
    
graph = nx.Graph()

nb_nodes = len(data['links'])

# Construct the graph edge by edge (there has to be a nicer way)
for i in range(0, nb_nodes-1):
    graph.add_edge(data['links'][i]['source'], data['links'][i]['target'])

print("Number of nodes:", nx.number_of_nodes(graph))
print("Number of edges:", nx.number_of_edges(graph))

Number of nodes: 26481
Number of edges: 43436


In [43]:
# Build the continuous-time SIR epidemic
sir = epidemics_helper.SimulationSIR(graph, beta=10.0, gamma=0.1)

# Launch epidemic
sir.launch_epidemic(source=23654, max_time=100.0)

Epidemic stopped after 106.40 days | 106.40 days elapsed | 0.1% susceptible, 0.0% infected, 99.9% recovered


In [64]:
# plotting values
plt.plot(sorted(sir.inf_time))
plt.plot(sorted(sir.rec_time))

# finding number of infections, susceptible and recovered
nb_susc = nb_nodes
nb_inf = 0
nb_rec = 0

# on doit plot nb d'infectés ect en fonction du temps, donc on doit regarder pour chaque temps le nombre d'inf ect? 
# Idée: sort les inf et rec times 

for i in range(0, nb_nodes-1):
    if(sir.inf_time[i] == inf):
        nb_inf += 1
    else if(sir.rec_time[i] != inf)
        nb_rec += 1
        nb_inf -= 1

SyntaxError: unexpected EOF while parsing (<ipython-input-64-37d550f60de9>, line 9)

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

#### Exercise 2.11: Strategy 2